In [103]:
#!pip install nest_asyncio

In [104]:
import requests
import urllib
import pandas as pd
from requests_html import HTML
from requests_html import HTMLSession

from bs4 import BeautifulSoup
import re
import time

In [105]:
def get_source(url):
    """Return the source code for the provided URL. 
    Args: 
        url (string): URL of the page to scrape.
    Returns:
        response (object): HTTP response object from requests_html. 
    """
    time.sleep(5) # prevents captcha hopefully

    try:
        session = HTMLSession()
        response = session.get(url)
        
        if response.status_code == 429:
            raise requests.exceptions.RequestException(f"Too many requests have been made. Stopped at url:{url}")
        
        return response

    except requests.exceptions.RequestException as e:
        print(e)

In [106]:
def get_author_profile(author_name):
    
    # get HTML response for author
    author_name = author_name.replace("ä", "ae").replace("ö", "oe").replace("ü", "ue").replace("ß", "ss")
    query = author_name.replace(" ", "-")
    response = get_source("https://www.researchgate.net/profile/" + query)
    soup = BeautifulSoup(response.html.raw_html)
        
    return soup

In [107]:
def get_author_publication_list(author_name, page):
    
    # get HTML response for author
    author_name = author_name.replace("ä", "ae").replace("ö", "oe").replace("ü", "ue").replace("ß", "ss")
    query = author_name.replace(" ", "+")
    response = get_source(f"https://www.researchgate.net/search/publication?q=\"{query}\"&page={page}")
    soup = BeautifulSoup(response.html.raw_html)
    
    return soup

In [108]:
def get_all_publications(page):
    return page.find_all("div", class_="nova-legacy-v-publication-item__body")

In [109]:
def get_publisher(publication_link, publication_type):
    
    publication_profile = BeautifulSoup(get_source(publication_link).html.raw_html)
    
    metadata = publication_profile.find("div", class_="research-detail-header-section__metadata")

    publisher = None
    
    if not metadata: # don't search for publisher if no metadata
        pass
    
    elif publication_type == 'Article':

        if metadata.find("a", class_="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-decorated"):
            publisher = metadata.find("a", class_="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-decorated").text

    elif publication_type in ['Book', 'Chapter']:

        if metadata.find("li", class_="nova-legacy-e-list__item"):

            # there are multiple items with that class
            for meta in metadata.find_all("li", class_="nova-legacy-e-list__item"):
                if "Publisher" in meta.text:
                    publisher = meta.text.split("Publisher: ",1)[1]

    elif publication_type in ['Poster', 'Conference Paper']:

        if metadata.find("li", class_="nova-legacy-e-list__item"):

            # there are multiple items with that class
            for meta in metadata.find_all("li", class_="nova-legacy-e-list__item"):
                if "Conference" in meta.text:
                    publisher = meta.text.split("Conference: ",1)[1]
                    
    return publisher

In [110]:
def get_publication_info(publication):
    
    publication_title = ""
    publication_year = ""
    authors_list = []
    publication_type = ""
    publisher = ""
    
    # publication title
    if publication.find("div", class_="nova-legacy-e-text nova-legacy-e-text--size-l nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit nova-legacy-v-publication-item__title"):
        publication_title = publication.find("div", class_="nova-legacy-e-text nova-legacy-e-text--size-l nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit nova-legacy-v-publication-item__title").string

    # publication year
    if publication.find("li", class_="nova-legacy-e-list__item nova-legacy-v-publication-item__meta-data-item"):
        publication_year_string = publication.find("li", class_="nova-legacy-e-list__item nova-legacy-v-publication-item__meta-data-item").string
        publication_year = re.search(r"([0-9]{4})", publication_year_string).group(1) # only get year
    
    # authors
    if publication.find("span", class_="nova-legacy-v-person-inline-item__fullname"):
        publication_author_list = publication.find_all("span", class_="nova-legacy-v-person-inline-item__fullname")
        authors_list = [x.text for x in publication_author_list]
    
    # publication type
    if publication.find("span", class_= "nova-legacy-e-badge nova-legacy-e-badge--color-green nova-legacy-e-badge--display-block nova-legacy-e-badge--luminosity-high nova-legacy-e-badge--size-l nova-legacy-e-badge--theme-solid nova-legacy-e-badge--radius-m nova-legacy-v-publication-item__badge"):
        publication_type = publication.find("span", class_= "nova-legacy-e-badge nova-legacy-e-badge--color-green nova-legacy-e-badge--display-block nova-legacy-e-badge--luminosity-high nova-legacy-e-badge--size-l nova-legacy-e-badge--theme-solid nova-legacy-e-badge--radius-m nova-legacy-v-publication-item__badge").text
    
    # publisher
    if publication.find("div", class_="nova-legacy-e-text nova-legacy-e-text--size-l nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit nova-legacy-v-publication-item__title")\
    .find("a", class_="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-bare"):

        publication_link = publication.find("div", class_="nova-legacy-e-text nova-legacy-e-text--size-l nova-legacy-e-text--family-sans-serif nova-legacy-e-text--spacing-none nova-legacy-e-text--color-inherit nova-legacy-v-publication-item__title")\
        .find("a", class_="nova-legacy-e-link nova-legacy-e-link--color-inherit nova-legacy-e-link--theme-bare")['href']
        
        if not publication_link.startswith("https://www.researchgate.net/"):
            publication_link = "https://www.researchgate.net/" + publication_link
        
        publisher = get_publisher(publication_link, publication_type)
    
    return publication_title, publication_year, authors_list, publication_type, publisher

In [111]:
def get_dataframe_with_publications(author_name):
    author_profile = get_author_profile(author_name)
    
    publication_titles = []
    publication_years = []
    authors_lists = []
    publication_types = []
    publisher_list = []
    author_name_list = []

    
    if author_profile:
        all_publication_entries = get_all_publications(author_profile)
        
    for publication_entry in all_publication_entries:
        publication_title, publication_year, authors_list, publication_type, publisher = get_publication_info(publication_entry)

        publication_titles.append(publication_title)
        publication_years.append(publication_year)
        authors_lists.append(authors_list)
        publication_types.append(publication_type)
        author_name_list.append(author_name)
        publisher_list.append(publisher)
    
    
    data = {
            'AUTHOR_NAME': author_name_list,
            'PUB_TITLE': publication_titles, 
            'PUB_YEAR': publication_years, 
            'PUB_AUTHORS': authors_lists,  
            'PUB_TYPE': publication_types,
            'PUB_PUBLISHER': publisher_list
           }
    
    return pd.DataFrame(data)

In [112]:
def get_dataframe_with_publications_no_profile(author_name):
    
    publication_titles = []
    publication_years = []
    authors_lists = []
    publication_types = []
    publisher_list = []
    author_name_list = []
    
    keep_going = True
    page_number = 1
    
    while keep_going:
        page_with_publications = get_author_publication_list(author_name, page_number)
        page_number = page_number + 1 # go to next page
        
        all_publication_entries = get_all_publications(page_with_publications)
        
        if not all_publication_entries:
            keep_going = False
            continue
        
        for publication_entry in all_publication_entries:
            publication_title, publication_year, authors_list, publication_type, publisher = get_publication_info(publication_entry)
        
            publication_titles.append(publication_title)
            publication_years.append(publication_year)
            authors_lists.append(authors_list)
            publication_types.append(publication_type)
            author_name_list.append(author_name)
            publisher_list.append(publisher)
    
    
    data = {
            'AUTHOR_NAME': author_name_list,
            'PUB_TITLE': publication_titles, 
            'PUB_YEAR': publication_years, 
            'PUB_AUTHORS': authors_lists,  
            'PUB_TYPE': publication_types,
            'PUB_PUBLISHER': publisher_list
           }
    
    return pd.DataFrame(data)

In [113]:
# get HTML response for author
#author_name = "Stephan Daurer"
author_names = ["Katerina Matysova", "Gerhard Hellstern", "Michael Bächle", "Thomas Asche", "Oliver Bährle", "Wolfgang Bihler"]

list_of_dicts_with_result = pd.DataFrame(columns=['PUB_TITLE', 'PUB_YEAR', 'PUB_AUTHORS', 'PUB_TYPE', 'PUB_PUBLISHER', 'AUTHOR_NAME'])

for author_name in author_names:
    pd_publications = get_dataframe_with_publications(author_name)
    
    if pd_publications.empty:
        pd_publications = get_dataframe_with_publications_no_profile(author_name)
        
        if pd_publications.empty:
            print(f"No publications found for {author_name}.")
            
    list_of_dicts_with_result = list_of_dicts_with_result.append(pd_publications)
        

Too many requests have been made. Stopped at url:https://www.researchgate.net/search/publication?q="Gerhard+Hellstern"&page=3


AttributeError: 'NoneType' object has no attribute 'html'

In [ ]:
pd_publications

In [ ]:
list_of_dicts_with_result.append(pd_publications)
# 9:40 Uhr